<a href="https://colab.research.google.com/github/MaTinABoL/ChatGPT-voice-assistant/blob/main/ChatGPT%20voice%20assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git

In [ ]:
!pip install gtts

In [ ]:
from IPython.display import HTML, Javascript
from google.colab.output import eval_js
import base64

In [ ]:
js = Javascript("""
  async function recordAudio(){
        const div = document.createElement('div');
        const audio = document.createElement('audio');
        const startButton = document.createElement('button');
        const stopButton = document.createElement('button');

        startButton.textContent = 'Start Recording';
        stopButton.textContent = 'Stop Recording';
        document.body.appendChild(div);
        div.appendChild(startButton);
        div.appendChild(stopButton);
        div.appendChild(audio);

        const stream = await navigator.mediaDevices.getUserMedia({audio: true});
        let recorder = new MediaRecorder(stream);
        audio.style.display = 'block';
        audio.srcObject = stream;
        audio.controls = true;
        audio.muted = true;

        await new Promise((resolve) => startButton.onclick = resolve);
        startButton.replaceWith(stopButton);
        recorder.start();

        await new Promise((resolve) => stopButton.onclick = resolve);
        recorder.stop();

        let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
        let arrBuff = await recData.data.arrayBuffer();
        stream.getAudioTracks()[0].stop();
        div.remove();

        let binaryString = '';
        let bytes = new Uint8Array(arrBuff);
        bytes.forEach((byte) => { binaryString += String.fromCharCode(byte) });

        const url = URL.createObjectURL(recData.data);
        const player = document.createElement('audio');
        player.controls = true;
        player.src = url;
        document.body.appendChild(player);

        return btoa(binaryString);
    };
""")

output = display(js, display_id=True)


<IPython.core.display.Javascript object>

In [ ]:
display(js)
output = eval_js('recordAudio({})')
with open("audio.mp3","wb") as file:
   binary = base64.b64decode(output)
   file.write(binary)
print("Recording saved to:",file.name)

<IPython.core.display.Javascript object>

Recording saved to: audio.mp3


In [ ]:
import whisper
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:45<00:00, 67.9MiB/s]


In [ ]:
result = model.transcribe("/content/audio.mp3")

x = result["text"]
lang = result["language"]

print(x)


 Answer briefly what do you know about Harvard University?


In [ ]:
import openai

API_KEY = "YOUR API KEY"
openai.api_key = API_KEY

response = openai.chat.completions.create(model="gpt-3.5-turbo",messages=[{"role":"user",f"content":x}])

print(response.choices[0].message.content)

response_text = response.choices[0].message.content

Harvard University is a prestigious Ivy League institution located in Cambridge, Massachusetts, United States. It was established in 1636 and is one of the oldest universities in the country. Harvard is renowned for its academic excellence and has a wide range of programs across various disciplines, including humanities, social sciences, natural sciences, engineering, and professional fields. The university is known for its highly selective admissions process and its notable alumni, including successful entrepreneurs, politicians, scientists, and celebrities. Harvard is also home to various esteemed research centers and libraries, such as the Harvard Library, which is the largest private library system in the world.


In [ ]:
from gtts import gTTS
from IPython.display import Audio

file = "./audio1.mp3"

gTTS (text = response_text,lang = lang).save(file)


wn = Audio("./audio1.mp3", autoplay=True)
display(wn)
